<a href="https://colab.research.google.com/github/coffeemountain/kaggle_otto/blob/main/notebook/kohiyama_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import numpy as np
import pandas as pd
import collections
from collections import Counter
import lightgbm as lgb
import pickle
import glob
import gc
import sys
from tqdm.notebook import tqdm

In [4]:
%cd /content/drive/MyDrive/kaggle/2022/OTTO/share
! pip install import-ipynb
import import_ipynb
from utils import TrainConfig, Dataset, reduce_mem_usage

/content/drive/.shortcut-targets-by-id/1H8a2MOay2LUWBmYLaUl2SUzTIGsFdpHT/share
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00
importing Jupyter notebook from utils.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 2.2 MB/s eta 0:00:00
Traceback (most recent call last):
  File "rapidsai-csp-utils/colab/pip-install.py", line 17, in <module>
    pynvml.nvmlInit()
  File "/usr/local/lib/python3.8/dist-packages/pynvml/nvml.py", line 1450, in nvmlInit
    nvmlInitWithFlags(0)
  File "/usr/local/lib/python3.8/dist-packages/pynvml/nvml.py", line 1440, in nvmlInitWith

In [1]:
DATASET_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/share/example_train2'

configs = [
    # CLICK用設定
    TrainConfig(
        target='clicks',
        # CV
        cv_train_candidates_file=DATASET_DIR + '/candidates/v2/cv_clicks_train_candidates.parquet',
        cv_train_labels_file=DATASET_DIR + '/input/train_labels.parquet',
        cv_test_candidates_file=DATASET_DIR + '/candidates/v2/cv_clicks_test_candidates.parquet',
        cv_test_labels_file=DATASET_DIR + '/input/test_labels.parquet',
        cv_features_dir=DATASET_DIR + '/features/cv',
        cv_output_file=DATASET_DIR + '/output/clicks_cv_v2.bst',

        # LB
        lb_train_candidates_file=DATASET_DIR + '/candidates/v2/lb_clicks_train_candidates.parquet',
        lb_train_labels_file=DATASET_DIR + '/input/lb_train_labels.parquet',
        lb_test_candidates_file=DATASET_DIR + '/candidates/v2/lb_clicks_test_candidates.parquet',
        lb_features_dir=DATASET_DIR + '/features/lb',
        lb_output_file=DATASET_DIR + '/output/clicks_lb_v2.bst',
        features=[
          'item_item_count',
          'item_user_count',
          'item_buy_ratio',
          'items_count_by_session_clicked',
          'items_count_by_session_carted',
          'items_count_by_session_orderd',
          'aid_init_clicked_train_date',
          'aid_init_carted_train_date',
          'aid_init_ordered_train_date',
          # 'aid_train_clicked_times',
          # 'aid_train_carted_times',
          # 'aid_train_ordered_times'
        ],
        params={
          'objective': 'lambdarank',
          'metric': 'ndcg', #
          'lambdarank_truncation_level': 10,
          'ndcg_eval_at': 20,
          'n_estimators': 10,
          'boosting_type': 'dart',
          'random_state': 0,
        },
        submission_output_file=DATASET_DIR + '/output/submissions/clicks_v2.csv'
    ),
        TrainConfig(
        target='carts',
        # CV
        cv_train_candidates_file=DATASET_DIR + '/candidates/v2/cv_clicks_train_candidates.parquet',
        cv_train_labels_file=DATASET_DIR + '/input/train_labels.parquet',
        cv_test_candidates_file=DATASET_DIR + '/candidates/v2/cv_clicks_test_candidates.parquet',
        cv_test_labels_file=DATASET_DIR + '/input/test_labels.parquet',
        cv_features_dir=DATASET_DIR + '/features/cv',
        cv_output_file=DATASET_DIR + '/output/carts_cv_v2.bst',

        # LB
        lb_train_candidates_file=DATASET_DIR + '/candidates/v2/lb_clicks_train_candidates.parquet',
        lb_train_labels_file=DATASET_DIR + '/input/lb_train_labels.parquet',
        lb_test_candidates_file=DATASET_DIR + '/candidates/v2/lb_clicks_test_candidates.parquet',
        lb_features_dir=DATASET_DIR + '/features/lb',
        lb_output_file=DATASET_DIR + '/output/carts_lb_v2.bst',
        features=[
          'item_item_count',
          'item_user_count',
          'item_buy_ratio',
          'items_count_by_session_clicked',
          'items_count_by_session_carted',
          'items_count_by_session_orderd',
          'aid_init_clicked_train_date',
          'aid_init_carted_train_date',
          'aid_init_ordered_train_date',
          # 'aid_train_clicked_times',
          # 'aid_train_carted_times',
          # 'aid_train_ordered_times'
        ],
        params={
          'objective': 'lambdarank',
          'metric': 'ndcg', #
          'lambdarank_truncation_level': 10,
          'ndcg_eval_at': 20,
          'n_estimators': 10,
          'boosting_type': 'dart',
          'random_state': 0,
        },
        submission_output_file=DATASET_DIR + '/output/submissions/carts_v2.csv'
    ),
        TrainConfig(
        target='orders',
        # CV
        cv_train_candidates_file=DATASET_DIR + '/candidates/v2/cv_clicks_train_candidates.parquet',
        cv_train_labels_file=DATASET_DIR + '/input/train_labels.parquet',
        cv_test_candidates_file=DATASET_DIR + '/candidates/v2/cv_clicks_test_candidates.parquet',
        cv_test_labels_file=DATASET_DIR + '/input/test_labels.parquet',
        cv_features_dir=DATASET_DIR + '/features/cv',
        cv_output_file=DATASET_DIR + '/output/orders_cv_v2.bst',

        # LB
        lb_train_candidates_file=DATASET_DIR + '/candidates/v2/lb_clicks_train_candidates.parquet',
        lb_train_labels_file=DATASET_DIR + '/input/lb_train_labels.parquet',
        lb_test_candidates_file=DATASET_DIR + '/candidates/v2/lb_clicks_test_candidates.parquet',
        lb_features_dir=DATASET_DIR + '/features/lb',
        lb_output_file=DATASET_DIR + '/output/orders_lb_v2.bst',
        features=[
          'item_item_count',
          'item_user_count',
          'item_buy_ratio',
          'items_count_by_session_clicked',
          'items_count_by_session_carted',
          'items_count_by_session_orderd',
          'aid_init_clicked_train_date',
          'aid_init_carted_train_date',
          'aid_init_ordered_train_date',
          # 'aid_train_clicked_times',
          # 'aid_train_carted_times',
          # 'aid_train_ordered_times'
        ],
        params={
          'objective': 'lambdarank',
          'metric': 'ndcg', #
          'lambdarank_truncation_level': 10,
          'ndcg_eval_at': 20,
          'n_estimators': 10,
          'boosting_type': 'dart',
          'random_state': 0,
        },
        submission_output_file=DATASET_DIR + '/output/submissions/orders_v2.csv'
    )
]

NameError: ignored

In [ ]:
def calc_recall(test_labels, pred_labels):
  test_labels = test_labels.merge(pred_labels, how='left', on='session')

  if test_labels['labels'].isnull().any():
    message = '推論できていないデータがあります。'
    raise Exception(message)

  test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1) 
  test_labels['gt_count'] = test_labels['ground_truth'].str.len().clip(0, 20)
  recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()

  return recall

def evaluate(test_labels_file, pred_labels, target):
  recall = calc_recall(test_labels_target, pred_labels)
  return recall

In [ ]:
def train(train_data, features, target, params):

  X_train = train_data[features]
  y_train = train_data[target]
  group_train = train_data.groupby('session')['aid'].agg('count').values

  dtrain = lgb.Dataset(X_train, y_train, group=group_train)
  model = lgb.train(params, dtrain)

  del X_train, y_train, group_train, dtrain
  gc.collect()

  return model

def predict(test_data, features, model):
  X_test = test_data[features]

  # 各candidateに対して、scoreを計算
  test_data['score'] = model.predict(X_test)

  pred_df = test_data[['session', 'aid', 'score']]

  # session(user)毎に、scoreが高い順に並べて、ランキング情報を付与
  pred_df['rank'] = pred_df.sort_values(['session','score'], ascending=[True, False]).groupby('session').cumcount().astype('int8') + 1

  # 各session 1~20位までのデータをlist化
  pred_df = pred_df.groupby('session')['aid'].apply(list).to_frame().reset_index()
  pred_df.rename(columns={'aid': 'labels'}, inplace=True)

  return pred_df[['session', 'labels']]

In [ ]:
# config設定たち
for config in configs:
  # ==================== CVモデル学習 =======================
  cv_train_data = Dataset().load_train(
    candidates_file=config.cv_train_candidates_file,
    features_dir=config.cv_features_dir,
    features=config.features,
    labels_file=config.cv_train_labels_file,
    target=config.target,
  )

  model = train(cv_train_data, config.features, config.target, config.params)
  model.save_model(config.cv_output_file)

  del cv_train_data
  gc.collect()
  # =======================================================

  # ==================== CVモデル予測 ========================
  cv_test_data = Dataset().load_test(
    candidates_file=config.cv_test_candidates_file,
    features_dir=config.cv_features_dir,
    features=config.features,
  )

  pred_labels = predict(cv_test_data, config.features, model)
  display(pred_labels)

  del cv_test_data
  # =======================================================

  # ==================== CVモデル評価 ========================
  test_labels = pd.read_parquet(config.cv_test_labels_file)
  test_labels_target = test_labels[test_labels['type'] == config.target]
  result = evaluate(test_labels_target, pred_labels, config.target)
  print(f'CV recall@20: {result}')
  # =======================================================

  # ==================== LBモデル学習 ========================
  lb_train_data = Dataset().load_train(
    candidates_file=config.lb_train_candidates_file,
    features_dir=config.lb_features_dir,
    features=config.features,
    labels_file=config.lb_train_labels_file,
    target=config.target,
  )

  lb_model = train(lb_train_data, config.features, config.target, config.params)
  lb_model.save_model(config.lb_output_file)

  del lb_train_data
  gc.collect()
  # =======================================================

  # ==================== LBモデル予測 ========================
  lb_test_data = Dataset().load_test(
    candidates_file=config.lb_test_candidates_file,
    features_dir=config.lb_features_dir,
    features=config.features,
  )

  pred_labels = predict(lb_test_data, config.features, lb_model)
  display(pred_labels)

  # submission形式の保存
  pred_labels['session_type'] = pred_labels['session'].apply(lambda session: f'{session}_{config.target}')
  pred_labels['labels'] = pred_labels['labels'].apply(lambda labels: ' '.join([str(label) for label in labels]))
  pred_labels[['session_type', 'labels']].to_csv(config.submission_output_file)
  # =======================================================

1. prepare candidates
2. prepare features


  0%|          | 0/9 [00:00<?, ?it/s]

check
check
check
check
check
check
check
check
check
3. prepare labels


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


1. prepare candidates
2. prepare features


...

1. prepare candidates
2. prepare features


  0%|          | 0/9 [00:00<?, ?it/s]

check
check
check
check
check
check
check
check
check


<ipython-input-7-4de6e27b3824>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df['rank'] = pred_df.sort_values(['session','score'], ascending=[True, False]).groupby('session').cumcount().astype('int8') + 1


,session,labels
0,12899918,"[1640070, 785544, 875146, 713211, 758415, 2244..."
1,12900181,[25391]
2,12900290,"[298888, 1141641, 276895, 1768485, 554917, 911..."
3,12900387,[937472]
4,12900424,"[1321891, 351300, 97414, 1252113, 1514520, 218..."
...,...,...
16713,14571383,"[506816, 992452, 1474022, 1578470, 85735, 5014..."
16714,14571410,"[1345697, 792865, 1523782, 392774, 296, 131107..."
16715,14571443,"[279041, 164613, 712456, 1648911, 1414544, 238..."
16716,14571492,"[1184000, 95488, 1841505, 1497313, 659686, 633..."


In [ ]:
# LB提出用
output_filename = '/output/submissions/v1_all.csv'

# Config (作成したファイルが３つだったら、提出用のsubmissionも作る)
if len(configs) != 3:
  break

dfs = []
for config in configs:
  df = pd.read_csv(config.submission_output_file)
  dfs.append(df)

submission_df = pd.concat(dfs)

submission_df.to_csv(output_filename)